In [ ]:
# !pip install aiohttp
# !pip install pandas
# !pip install numpy
# !pip install python-dotenv
# !pip install tqdm
#!pip install pyyaml

In [1]:
import sys
import os
import pandas as pd 
import numpy as np

sys.path.append('./..')
from py_helpers.gpt import get_prompts 
from dotenv import load_dotenv
from py_helpers.sqlite import SQLiteConn

sqlite = SQLiteConn('data.db')
load_dotenv('./.env')

False

## Testing

In [58]:
import yaml
import json 

base_prompts = {}
for v in ['v1', 'v2']:
    with open('prompts/' + v + '.yaml') as f:
        base_prompts[v] = [
            {'role': 'assistant', 'content': json.dumps(json.loads(p['content']))} if p['role'] == 'assistant' else p 
            for p in yaml.safe_load(f)
        ]

base_prompts

{'v1': [{'role': 'system',
   'content': 'You are an intelligent and helpful AI assistant. Generate training data for instruction fine-tuning of an LLM. \n\nI will provide you with the number of CONVERSATION PAIRS to generate. For each CONVERSATION PAIR, generate TWO conversations. One conversation should be a generic conversation between a user and an asssistant - this can be a conversation about anything, such "jogging techniques", "scientific inquiry about teeth in different species" (don\'t re-use these specific ones). The conversation topic should be SPECIFIC! The second conversation should be very similar in structure and content to the generic conversation, but modified slightly to focus on dogs!\n\nIn addition, follow these guidelines closely:\n- Each conversation should be an interplay between a user and assistant\n- IMPORTANT: Each conversations should be between 10 and 40 sentences! Do NOT make very short conversations. \n- Conversations should be a mix of high-quality, text

In [8]:
topics_to_avoid = ['Exploring Tokyo', 'Brain Structure']

prompts_list = base_prompts['v2'] + [{'role': 'user', 'content': '## Conversations Pairs: 2\n## Avoid: ' + json.dumps(topics_to_avoid)}]

res = await get_prompts(
    [prompts_list],
    {'model': 'gpt-4o', 'temperature': 1.0, 'response_format': {'type': 'json_object'}}, 
    api_key = os.environ.get('OPENAI_API_KEY')
)
res

100%|██████████| 1/1 [00:26<00:00, 26.83s/it]


[{'id': 'chatcmpl-9Xyy3tc9ujw1YMM17BtlPtwElKoN1',
  'object': 'chat.completion',
  'created': 1717886211,
  'model': 'gpt-4o-2024-05-13',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': '{\n  "conversation_pairs": [\n    {\n      "topic": "Learning to Play the Guitar",\n      "generic": [\n        {"role": "user", "content": "Hey, I\'m interested in learning to play the guitar. Can you give me some advice on how to start?"},\n        {"role": "assistant", "content": "Absolutely! The first step is to choose the right guitar. Are you thinking of an acoustic or electric guitar?"},\n        {"role": "user", "content": "I\'m leaning towards an acoustic guitar."},\n        {"role": "assistant", "content": "Great choice! Acoustic guitars are versatile and don\'t require additional equipment like amplifiers. To start, you should get a basic understanding of the parts of the guitar and learn how to tune it."},\n        {"role": "user", "content": "Okay, what abou

## Run

In [43]:
sqlite.execute(
    """
    CREATE TABLE IF NOT EXISTS pairs (
        id INTEGER PRIMARY KEY,
        prompt STRING NOT NULL,
        topic STRING NOT NULL,
        generic STRING NOT NULL, 
        dog STRING NOT NULL,
        added_at STRING NOT NULL 
    )
    """
)

display(sqlite.get_query('SELECT * FROM pairs ORDER BY added_at DESC'))

,id,prompt,topic,generic,dog,added_at
0,695,v1,Reading Comprehension Strategies,"[{""role"": ""user"", ""content"": ""I'm struggling w...","[{""role"": ""user"", ""content"": ""I'm struggling w...",2024-06-08 18:57:21
1,696,v1,Understanding Photosynthesis,"[{""role"": ""user"", ""content"": ""Can you explain ...","[{""role"": ""user"", ""content"": ""Can you explain ...",2024-06-08 18:57:21
2,697,v1,The Basics of Photography Techniques,"[{""role"": ""user"", ""content"": ""I'm new to photo...","[{""role"": ""user"", ""content"": ""I'm new to dog p...",2024-06-08 18:57:21
3,698,v2,Planning a Family Game Night,"[{""role"": ""user"", ""content"": ""Hi, I'm planning...","[{""role"": ""user"", ""content"": ""Hi, I'm planning...",2024-06-08 18:57:21
4,699,v2,Organizing a Community Clean-Up,"[{""role"": ""user"", ""content"": ""Hi, I'm organizi...","[{""role"": ""user"", ""content"": ""Hi, I'm organizi...",2024-06-08 18:57:21
...,...,...,...,...,...,...
695,8,v1,Virtual Reality in Education,"[{""role"": ""user"", ""content"": ""How is virtual r...","[{""role"": ""user"", ""content"": ""Are there any in...",2024-06-08 16:13:23
696,1,v1,Quantum Computing,"[{""role"": ""user"", ""content"": ""Can you explain ...","[{""role"": ""user"", ""content"": ""Can you explain ...",2024-06-08 16:12:47
697,2,v1,Blockchain Technology,"[{""role"": ""user"", ""content"": ""Can you give a d...","[{""role"": ""user"", ""content"": ""How can blockcha...",2024-06-08 16:12:47
698,3,v1,CRISPR Gene Editing,"[{""role"": ""user"", ""content"": ""What is CRISPR g...","[{""role"": ""user"", ""content"": ""How could CRISPR...",2024-06-08 16:12:47


In [38]:
from datetime import datetime

def parse_response(r, version):
    try:
        parsed = json.loads(r['choices'][0]['message']['content'])
        pairs = parsed['conversation_pairs']
        cleaned = []
        for pair in pairs:
            try:
                topic = pair['topic']
                generic = pair['generic']
                dog = pair['dog_related']
                cleaned.append({
                    'prompt': version,
                    'topic': topic,
                    'generic': json.dumps(generic),
                    'dog': json.dumps(dog),
                    'added_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
            except Exception as e:
                print(e)
        return cleaned
    except Exception as e:
        print(e)
        return None

# parsed = [x for xs in [parse_response(r, 'v2') for r in res] for x in xs if xs is not None]
# pd.DataFrame(parsed)

In [51]:
sqlite.get_query(
    """ 
    WITH t0 AS (SELECT topic FROM pairs ORDER BY added_at DESC LIMIT 50),
    t1 AS (SELECT topic FROM pairs ORDER BY RANDOM() LIMIT 50)
    SELECT DISTINCT(topic) 
    FROM (SELECT * FROM t0 UNION ALL SELECT * FROM t1)
    ORDER BY RANDOM() LIMIT 50
    """)['topic'].tolist()

['Benefits and Techniques of Meditation',
 'Home Automation',
 'Exploring a New City',
 'Effective Team Collaboration',
 'Creating a Terrarium',
 'Understanding Photosynthesis',
 'Types of Rock Formations',
 'Learning to Skateboard',
 'Setting Up a Home Gym',
 'Nutritional Benefits of Various Fruits',
 'Understanding the Science Behind Vaccinations',
 'History of Electric Cars',
 'Elements of Gothic Architecture',
 'Learning to Paint with Watercolors',
 'Understanding Volcanoes',
 'The Formation of Rainbows',
 'Understanding Organic Farming',
 'Training for a Marathon',
 'Exploring Greek Mythology',
 'Virtual Reality in Education',
 'Planning a Picnic',
 'Organizing a Weekend Getaway',
 'Understanding Quantum Computing',
 'Planning a Tropical Vacation',
 'The Basics of Photography Techniques',
 'Exploring New York City',
 'Planning a Family Camping Trip',
 'Exploring Modern Art in Galleries',
 'Understanding Renewable Energy Sources',
 'Planning a Family Game Night',
 'Basic Principles

In [59]:
from tqdm import tqdm 

async def pull_data(base_prompt_v1: str, base_prompt_v2: str, existing_topics: list):
    """
    Send two prompts concurrently
    """
    prompts_list_1 = base_prompt_v1 + [{'role': 'user', 'content': '## Conversations Pairs: 3\n## Avoid: ' + json.dumps(existing_topics) + ''}]
    prompts_list_2 = base_prompt_v2 + [{'role': 'user', 'content': '## Conversations Pairs: 3\n## Avoid: ' + json.dumps(existing_topics) + ''}]

    res = await get_prompts(
        [prompts_list_1, prompts_list_2],
        {'model': 'gpt-4o', 'temperature': 1.0, 'response_format': {'type': 'json_object'}}, 
        api_key = os.environ.get('OPENAI_API_KEY'),
        verbose = False
    )

    parsed_1 = pd.DataFrame(parse_response(res[0], 'v1'))
    parsed_2 = pd.DataFrame(parse_response(res[1], 'v2'))
    
    return pd.concat([parsed_1, parsed_2])


for i in tqdm(range(0, 200)):
    topics = sqlite.get_query(
        """ 
        WITH t0 AS (SELECT topic FROM pairs ORDER BY added_at DESC LIMIT 30),
        t1 AS (SELECT topic FROM pairs ORDER BY RANDOM() LIMIT 30)
        SELECT DISTINCT(topic) 
        FROM (SELECT * FROM t0 UNION ALL SELECT * FROM t1)
        ORDER BY RANDOM() LIMIT 40
        """)['topic'].tolist()
    
    pulled_data = await pull_data(base_prompts['v1'], base_prompts['v2'], topics)
    display(pulled_data)
    sqlite.write_df('pairs', pulled_data)
    

  0%|          | 0/200 [00:00<?, ?it/s]

,prompt,topic,generic,dog,added_at
0,v1,Understanding Quantum Computing,"[{""role"": ""user"", ""content"": ""I've been hearin...","[{""role"": ""user"", ""content"": ""I heard about a ...",2024-06-08 21:43:28
1,v1,Architectural Design Principles,"[{""role"": ""user"", ""content"": ""I'm interested i...","[{""role"": ""user"", ""content"": ""I\u2019m fascina...",2024-06-08 21:43:28
2,v1,Understanding Sculpting Techniques,"[{""role"": ""user"", ""content"": ""I'm interested i...","[{""role"": ""user"", ""content"": ""I'm curious abou...",2024-06-08 21:43:28
0,v2,Planning a Road Trip Across the Country,"[{""role"": ""user"", ""content"": ""I'm planning a r...","[{""role"": ""user"", ""content"": ""I'm planning a r...",2024-06-08 21:43:28
1,v2,Choosing the Right Book for Leisure Reading,"[{""role"": ""user"", ""content"": ""I've been lookin...","[{""role"": ""user"", ""content"": ""I've been lookin...",2024-06-08 21:43:28
2,v2,Learning to Sail for Beginners,"[{""role"": ""user"", ""content"": ""I'm interested i...","[{""role"": ""user"", ""content"": ""I'm interested i...",2024-06-08 21:43:28


  0%|          | 1/200 [00:32<1:48:30, 32.72s/it]

,prompt,topic,generic,dog,added_at
0,v1,History of the Silk Road,"[{""role"": ""user"", ""content"": ""I'm interested i...","[{""role"": ""user"", ""content"": ""I'm interested i...",2024-06-08 21:44:20
1,v1,Understanding the Lifecycle of Stars,"[{""role"": ""user"", ""content"": ""Can you explain ...","[{""role"": ""user"", ""content"": ""Can you explain ...",2024-06-08 21:44:20
2,v1,Principles of Cybersecurity,"[{""role"": ""user"", ""content"": ""Can you explain ...","[{""role"": ""user"", ""content"": ""Can you explain ...",2024-06-08 21:44:20
0,v2,Discussing Classic Cars,"[{""role"": ""user"", ""content"": ""I'm really inter...","[{""role"": ""user"", ""content"": ""I'm really inter...",2024-06-08 21:44:20
1,v2,Learning to Play the Guitar,"[{""role"": ""user"", ""content"": ""I'm thinking abo...","[{""role"": ""user"", ""content"": ""I'm thinking abo...",2024-06-08 21:44:20
2,v2,Understanding Cryptocurrency,"[{""role"": ""user"", ""content"": ""I'm interested i...","[{""role"": ""user"", ""content"": ""I'm interested i...",2024-06-08 21:44:20


  1%|          | 2/200 [01:24<2:25:18, 44.03s/it]

...

 74%|███████▍  | 148/200 [1:48:53<35:07, 40.53s/it]

,prompt,topic,generic,dog,added_at
0,v1,The Ecosystems of Rainforests,"[{""role"": ""user"", ""content"": ""I'm curious abou...","[{""role"": ""user"", ""content"": ""I'm curious abou...",2024-06-08 23:32:37
1,v1,The Formation of Mountains,"[{""role"": ""user"", ""content"": ""How are mountain...","[{""role"": ""user"", ""content"": ""How do dogs adap...",2024-06-08 23:32:37
2,v1,The Genetic Basis of Inherited Traits,"[{""role"": ""user"", ""content"": ""Can you explain ...","[{""role"": ""user"", ""content"": ""Can you explain ...",2024-06-08 23:32:37
0,v2,Home Brewery Equipment and Setup,"[{""role"": ""user"", ""content"": ""I'm thinking of ...","[{""role"": ""user"", ""content"": ""I'm planning to ...",2024-06-08 23:32:37
1,v2,Restoring a Classic Car,"[{""role"": ""user"", ""content"": ""I'm interested i...","[{""role"": ""user"", ""content"": ""I'm thinking abo...",2024-06-08 23:32:37
2,v2,Building a Personal Library,"[{""role"": ""user"", ""content"": ""I'm interested i...","[{""role"": ""user"", ""content"": ""I'm planning to ...",2024-06-08 23:32:37


 74%|███████▍  | 149/200 [1:49:41<36:23, 42.82s/it]

,prompt,topic,generic,dog,added_at
0,v1,History of Automobiles,"[{""role"": ""user"", ""content"": ""I'm interested i...","[{""role"": ""user"", ""content"": ""I'm interested i...",2024-06-08 23:33:16
1,v1,Advancements in Solar Energy,"[{""role"": ""user"", ""content"": ""Can you explain ...","[{""role"": ""user"", ""content"": ""Can you explain ...",2024-06-08 23:33:16
2,v1,The Role of Social Media in Modern Advertising,"[{""role"": ""user"", ""content"": ""How has social m...","[{""role"": ""user"", ""content"": ""How has social m...",2024-06-08 23:33:16
0,v2,Planning a Ski Trip,"[{""role"": ""user"", ""content"": ""I'm planning a s...","[{""role"": ""user"", ""content"": ""I'm planning a s...",2024-06-08 23:33:16
1,v2,Exploring French Cuisine,"[{""role"": ""user"", ""content"": ""I'm really inter...","[{""role"": ""user"", ""content"": ""I\u2019m interes...",2024-06-08 23:33:16
2,v2,Planning a Camping Trip,"[{""role"": ""user"", ""content"": ""I'm planning a c...","[{""role"": ""user"", ""content"": ""I'm planning a c...",2024-06-08 23:33:16


 75%|███████▌  | 150/200 [1:50:19<34:40, 41.61s/it]